In [1]:
# importando bibliotecas conexão PDGT
import json
import argparse
import subprocess
import boto3
import time
from pyathena import connect
import pandas.io.sql as sqlio
import sys
from ydata_profiling import ProfileReport
from botocore import UNSIGNED
from botocore.config import Config
import boto3.session
from botocore import exceptions
from scipy.stats import chi2_contingency

/home/lucasmateus/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# importando bibliotecas para análise
import pandas as pd
from scipy import stats
import math
import statsmodels.stats.api as sms
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import psycopg2
import dask.dataframe as dd
from collections import defaultdict
from scipy.stats import shapiro
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import mode
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
import googlemaps
from datetime import datetime
from googlemaps.exceptions import ApiError

In [3]:
class CustomException(Exception):
    pass

In [4]:
# aws athena connection info

In [5]:
# aws athena connection function

In [7]:
# SETANDO API DO MAPS

#api_key = "token"

gmaps_client = googlemaps.Client(key=api_key)

NameError: name 'api_key' is not defined

In [ ]:
query_proporcoes = """
sql query
"""

In [ ]:
df_proporcoes = execute_athena_query(query_proporcoes)

In [ ]:
df_proporcoes['proporcao_homens'] = df_proporcoes['qtd_pacientes_homens'] / df_proporcoes['total_pacientes']
df_proporcoes['proporcao_mulheres'] = df_proporcoes['qtd_pacientes_mulheres'] / df_proporcoes['total_pacientes']

df_proporcoes

,regional,qtd_pacientes_homens,qtd_pacientes_mulheres,total_pacientes,proporcao_homens,proporcao_mulheres
0,ES,107660,196503,7612762,0.014142,0.025812
1,Norte,135914,250751,7612762,0.017853,0.032938
2,NE2,180588,362902,7612762,0.023722,0.047670
3,RJ,216168,404848,7612762,0.028395,0.053180
4,CO,231395,428580,7612762,0.030396,0.056298
5,MG,268464,485084,7612762,0.035265,0.063720
6,NE1,293109,564595,7612762,0.038502,0.074164
7,Sul,378680,669421,7612762,0.049743,0.087934
8,SP Interior,440010,748676,7612762,0.057799,0.098345
9,SP CAV,438208,811206,7612762,0.057562,0.106559


In [ ]:
#COCHRAN FORMULA FOR SAMPLE SIZE - POPULAÇÃO COMPLETA

def cochran_sample_size(confidence, estimated_proportion, margin_of_error):
    z = np.abs(stats.norm.ppf((1 - confidence) / 2))  # Calculate Z-score
    n = (z**2 * estimated_proportion * (1 - estimated_proportion)) / margin_of_error**2
    return n

# Example values
confidence_level = 0.99  # 99% confidence
estimated_proportion = 0.5  # Estimated proportion (50% for maximum sample size)
margin_of_error = 0.01  # Margin of error (1%)

# Calculate sample size
sample_size = cochran_sample_size(confidence_level, estimated_proportion, margin_of_error)

print(f"Required Sample Size: {int(sample_size)}")

Required Sample Size: 16587


In [ ]:
df_proporcoes['tamanho_amostra'] = sample_size
df_proporcoes['amostra_homens'] = (df_proporcoes['proporcao_homens'] * sample_size).apply(lambda x: math.ceil(x))
df_proporcoes['amostra_mulheres'] = (df_proporcoes['proporcao_mulheres'] * sample_size).apply(lambda x: math.ceil(x))

df_proporcoes

,regional,qtd_pacientes_homens,qtd_pacientes_mulheres,total_pacientes,proporcao_homens,proporcao_mulheres,tamanho_amostra,amostra_homens,amostra_mulheres
0,ES,107660,196503,7612762,0.014142,0.025812,16587.241503,235,429
1,Norte,135914,250751,7612762,0.017853,0.032938,16587.241503,297,547
2,NE2,180588,362902,7612762,0.023722,0.047670,16587.241503,394,791
3,RJ,216168,404848,7612762,0.028395,0.053180,16587.241503,472,883
4,CO,231395,428580,7612762,0.030396,0.056298,16587.241503,505,934
5,MG,268464,485084,7612762,0.035265,0.063720,16587.241503,585,1057
6,NE1,293109,564595,7612762,0.038502,0.074164,16587.241503,639,1231
7,Sul,378680,669421,7612762,0.049743,0.087934,16587.241503,826,1459
8,SP Interior,440010,748676,7612762,0.057799,0.098345,16587.241503,959,1632
9,SP CAV,438208,811206,7612762,0.057562,0.106559,16587.241503,955,1768


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - HOMENS

spcav_h = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_homens'].values[0]
spint_h = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_homens'].values[0]
sul_h = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_homens'].values[0]
ne1_h = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_homens'].values[0]
mg_h = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_homens'].values[0]
co_h = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_homens'].values[0]
rj_h = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_homens'].values[0]
ne2_h = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_homens'].values[0]
norte_h = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_homens'].values[0]
es_h = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_homens'].values[0]

print(f'Amostra homens SP CAV: {spcav_h}')
print(f'Amostra homens SP Int: {spint_h}')
print(f'Amostra homens Sul: {sul_h}')
print(f'Amostra homens NE1: {ne1_h}')
print(f'Amostra homens MG: {mg_h}')
print(f'Amostra homens CO: {co_h}')
print(f'Amostra homens RJ: {rj_h}')
print(f'Amostra homens NE2: {ne2_h}')
print(f'Amostra homens Norte: {norte_h}')
print(f'Amostra homens ES: {es_h}')

Amostra homens SP CAV: 955
Amostra homens SP Int: 959
Amostra homens Sul: 826
Amostra homens NE1: 639
Amostra homens MG: 585
Amostra homens CO: 505
Amostra homens RJ: 472
Amostra homens NE2: 394
Amostra homens Norte: 297
Amostra homens ES: 235


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - MULHERES

spcav_m = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_mulheres'].values[0]
spint_m = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_mulheres'].values[0]
sul_m = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_mulheres'].values[0]
ne1_m = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_mulheres'].values[0]
mg_m = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_mulheres'].values[0]
co_m = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_mulheres'].values[0]
rj_m = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_mulheres'].values[0]
ne2_m = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_mulheres'].values[0]
norte_m = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_mulheres'].values[0]
es_m = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_mulheres'].values[0]

print(f'Amostra mulheres SP CAV: {spcav_m}')
print(f'Amostra mulheres SP Int: {spint_m}')
print(f'Amostra mulheres Sul: {sul_m}')
print(f'Amostra mulheres NE1: {ne1_m}')
print(f'Amostra mulheres MG: {mg_m}')
print(f'Amostra mulheres CO: {co_m}')
print(f'Amostra mulheres RJ: {rj_m}')
print(f'Amostra mulheres NE2: {ne2_m}')
print(f'Amostra mulheres Norte: {norte_m}')
print(f'Amostra hommulheresens ES: {es_m}')

Amostra mulheres SP CAV: 1768
Amostra mulheres SP Int: 1632
Amostra mulheres Sul: 1459
Amostra mulheres NE1: 1231
Amostra mulheres MG: 1057
Amostra mulheres CO: 934
Amostra mulheres RJ: 883
Amostra mulheres NE2: 791
Amostra mulheres Norte: 547
Amostra hommulheresens ES: 429


In [19]:
#COCHRAN FORMULA FOR SAMPLE SIZE - PACIENTE PARTICULAR

def cochran_sample_size(confidence, estimated_proportion, margin_of_error):
    z = np.abs(stats.norm.ppf((1 - confidence) / 2))  # Calculate Z-score
    n = (z**2 * estimated_proportion * (1 - estimated_proportion)) / margin_of_error**2
    return n

# Example values
confidence_level = 0.90  # 99% confidence
estimated_proportion = 0.05  # Estimated proportion (50% for maximum sample size)
margin_of_error = 0.05  # Margin of error (1%)

# Calculate sample size
sample_size_part = cochran_sample_size(confidence_level, estimated_proportion, margin_of_error)

print(f"Tamanho da amostra necessária: {int(sample_size_part)} para {confidence_level*100:.0f}% de nível de confiança e {margin_of_error*100:.0f}% de margem de erro.")



Tamanho da amostra necessária: 270 para 90% de nível de confiança e 5% de margem de erro.


In [ ]:
df_proporcoes['tamanho_amostra_part'] = sample_size_part
df_proporcoes['amostra_homens_part'] = (df_proporcoes['proporcao_homens'] * sample_size_part).apply(lambda x: math.ceil(x))
df_proporcoes['amostra_mulheres_part'] = (df_proporcoes['proporcao_mulheres'] * sample_size_part).apply(lambda x: math.ceil(x))

df_proporcoes

,regional,qtd_pacientes_homens,qtd_pacientes_mulheres,total_pacientes,proporcao_homens,proporcao_mulheres,tamanho_amostra,amostra_homens,amostra_mulheres,tamanho_amostra_part,amostra_homens_part,amostra_mulheres_part
0,ES,107660,196503,7612762,0.014142,0.025812,16587.241503,235,429,3151.575885,45,82
1,Norte,135914,250751,7612762,0.017853,0.032938,16587.241503,297,547,3151.575885,57,104
2,NE2,180588,362902,7612762,0.023722,0.047670,16587.241503,394,791,3151.575885,75,151
3,RJ,216168,404848,7612762,0.028395,0.053180,16587.241503,472,883,3151.575885,90,168
4,CO,231395,428580,7612762,0.030396,0.056298,16587.241503,505,934,3151.575885,96,178
5,MG,268464,485084,7612762,0.035265,0.063720,16587.241503,585,1057,3151.575885,112,201
6,NE1,293109,564595,7612762,0.038502,0.074164,16587.241503,639,1231,3151.575885,122,234
7,Sul,378680,669421,7612762,0.049743,0.087934,16587.241503,826,1459,3151.575885,157,278
8,SP Interior,440010,748676,7612762,0.057799,0.098345,16587.241503,959,1632,3151.575885,183,310
9,SP CAV,438208,811206,7612762,0.057562,0.106559,16587.241503,955,1768,3151.575885,182,336


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - HOMENS

spcav_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_homens_part'].values[0]
spint_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_homens_part'].values[0]
sul_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_homens_part'].values[0]
ne1_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_homens_part'].values[0]
mg_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_homens_part'].values[0]
co_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_homens_part'].values[0]
rj_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_homens_part'].values[0]
ne2_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_homens_part'].values[0]
norte_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_homens_part'].values[0]
es_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_homens_part'].values[0]

print(f'Amostra homens SP CAV: {spcav_h_part}')
print(f'Amostra homens SP Int: {spint_h_part}')
print(f'Amostra homens Sul: {sul_h_part}')
print(f'Amostra homens NE1: {ne1_h_part}')
print(f'Amostra homens MG: {mg_h_part}')
print(f'Amostra homens CO: {co_h_part}')
print(f'Amostra homens RJ: {rj_h_part}')
print(f'Amostra homens NE2: {ne2_h_part}')
print(f'Amostra homens Norte: {norte_h_part}')
print(f'Amostra homens ES: {es_h_part}')

Amostra homens SP CAV: 182
Amostra homens SP Int: 183
Amostra homens Sul: 157
Amostra homens NE1: 122
Amostra homens MG: 112
Amostra homens CO: 96
Amostra homens RJ: 90
Amostra homens NE2: 75
Amostra homens Norte: 57
Amostra homens ES: 45


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - MULHERES

spcav_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_mulheres_part'].values[0]
spint_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_mulheres_part'].values[0]
sul_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_mulheres_part'].values[0]
ne1_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_mulheres_part'].values[0]
mg_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_mulheres_part'].values[0]
co_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_mulheres_part'].values[0]
rj_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_mulheres_part'].values[0]
ne2_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_mulheres_part'].values[0]
norte_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_mulheres_part'].values[0]
es_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_mulheres_part'].values[0]

print(f'Amostra mulheres SP CAV: {spcav_m_part}')
print(f'Amostra mulheres SP Int: {spint_m_part}')
print(f'Amostra mulheres Sul: {sul_m_part}')
print(f'Amostra mulheres NE1: {ne1_m_part}')
print(f'Amostra mulheres MG: {mg_m_part}')
print(f'Amostra mulheres CO: {co_m_part}')
print(f'Amostra mulheres RJ: {rj_m_part}')
print(f'Amostra mulheres NE2: {ne2_m_part}')
print(f'Amostra mulheres Norte: {norte_m_part}')
print(f'Amostra hommulheresens ES: {es_m_part}')

Amostra mulheres SP CAV: 336
Amostra mulheres SP Int: 310
Amostra mulheres Sul: 278
Amostra mulheres NE1: 234
Amostra mulheres MG: 201
Amostra mulheres CO: 178
Amostra mulheres RJ: 168
Amostra mulheres NE2: 151
Amostra mulheres Norte: 104
Amostra hommulheresens ES: 82


In [ ]:
query_pacientes = f"""
sql query
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE PACIENTES EM DF

df_pac = execute_athena_query(query_pacientes)

In [ ]:
df_pac.columns

Index(['id_paciente', 'cpfpaciente', 'min_dt_pg', 'max_dt_pg',
       'anos_utilizacao', 'meses_utilizacao', 'dias_ult_utilizacao',
       'meses_ult_utilizacao', 'estado', 'cidade', 'idade', 'sexo',
       'endereco_paciente', 'qtd_consultas', 'qtd_exam_proc', 'qtd_particular',
       'tt_consulta', 'tm_consulta', 'tt_exam_proc', 'qtd_orc_nao_executado',
       'tt_orc_nao_executado', 'tm_utilizacao', 'pac_particular',
       'ultimo_canal', 'qtd_canais', 'mudou_canal', 'regional',
       'nm_especialidade', 'tabela', 'unidade', 'endereco_unidade',
       'cidade_clinica', 'score_r', 'score_f', 'score_v'],
      dtype='object')

In [ ]:
# TRATATIVAS NA BASE DE PACIENTES

# REMOVER ACENTOS DAS CIDADES 
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_pac['cidade'] = df_pac['cidade'].apply(remove_accents)
df_pac['cidade_clinica'] = df_pac['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_pac['mesma_cidade'] = (df_pac['cidade'] == df_pac['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_pac['faixa_etaria'] = pd.cut(df_pac['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_pac['faixa_utilizacao'] = pd.cut(df_pac['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)

In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_pac['score_recencia'] = df_pac['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_pac['score_frequencia'] = df_pac['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_v_text'
df_pac['score_valor'] = df_pac['tm_utilizacao'].map(lambda x: translate_value_score(x))

In [ ]:
# Iterate through the DataFrame
for index, row in df_pac.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_pac.at[index, 'distance_text'] = distance_text
                df_pac.at[index, 'distance_value'] = distance_value
                df_pac.at[index, 'duration_text'] = duration_text
                df_pac.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error.")
            continue

Skipping row 468 due to 'NOT_FOUND' error.
Skipping row 1933 due to 'NOT_FOUND' error.
Skipping row 1971 due to 'NOT_FOUND' error.
Skipping row 2094 due to 'NOT_FOUND' error.
Skipping row 2290 due to 'NOT_FOUND' error.
Skipping row 2325 due to 'NOT_FOUND' error.
Skipping row 2326 due to 'NOT_FOUND' error.
Skipping row 4572 due to 'NOT_FOUND' error.
Skipping row 4783 due to 'NOT_FOUND' error.
Skipping row 4853 due to 'NOT_FOUND' error.
Skipping row 4867 due to 'NOT_FOUND' error.
Skipping row 5460 due to 'NOT_FOUND' error.
Skipping row 5682 due to 'NOT_FOUND' error.
Skipping row 6553 due to 'NOT_FOUND' error.
Skipping row 6554 due to 'NOT_FOUND' error.
Skipping row 6671 due to 'NOT_FOUND' error.
Skipping row 8734 due to 'NOT_FOUND' error.
Skipping row 10166 due to 'NOT_FOUND' error.
Skipping row 10663 due to 'NOT_FOUND' error.
Skipping row 10755 due to 'NOT_FOUND' error.
Skipping row 10927 due to 'NOT_FOUND' error.
Skipping row 11147 due to 'NOT_FOUND' error.
Skipping row 11352 due to 'N

In [ ]:
# MOSTRAR DF

df_pac

In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_pac.to_excel('bd_paciente_geo.xlsx', index=False, sheet_name='base')

In [ ]:
query_titular = f"""
sql query
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE TITULAR EM DF

df_tit = execute_athena_query(query_titular)

In [ ]:
# TRATATIVAS NA BASE DE TITULAR

# REMOVER ACENTOS DAS CIDADES 
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_tit['cidade'] = df_tit['cidade'].apply(remove_accents)
df_tit['cidade_clinica'] = df_tit['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_tit['mesma_cidade'] = (df_tit['cidade'] == df_tit['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_tit['faixa_etaria'] = pd.cut(df_tit['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_tit['faixa_utilizacao'] = pd.cut(df_tit['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)

In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_tit['score_recencia'] = df_tit['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_tit['score_frequencia'] = df_tit['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_v_text'
df_tit['score_valor'] = df_tit['tm_utilizacao'].map(lambda x: translate_value_score(x))


In [ ]:
# Iterate through the DataFrame
for index, row in df_tit.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_tit.at[index, 'distance_text'] = distance_text
                df_tit.at[index, 'distance_value'] = distance_value
                df_tit.at[index, 'duration_text'] = duration_text
                df_tit.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error at {current_time}.")
            continue

Skipping row 27 due to 'NOT_FOUND' error at 2023-10-13 17:52:10.
Skipping row 818 due to 'NOT_FOUND' error at 2023-10-13 18:00:00.
Skipping row 1943 due to 'NOT_FOUND' error at 2023-10-13 18:10:56.
Skipping row 2667 due to 'NOT_FOUND' error at 2023-10-13 18:17:27.
Skipping row 2674 due to 'NOT_FOUND' error at 2023-10-13 18:17:31.
Skipping row 2675 due to 'NOT_FOUND' error at 2023-10-13 18:17:32.
Skipping row 2770 due to 'NOT_FOUND' error at 2023-10-13 18:18:27.
Skipping row 2886 due to 'NOT_FOUND' error at 2023-10-13 18:19:36.
Skipping row 3162 due to 'NOT_FOUND' error at 2023-10-13 18:22:16.
Skipping row 3455 due to 'NOT_FOUND' error at 2023-10-13 18:25:04.
Skipping row 3636 due to 'NOT_FOUND' error at 2023-10-13 18:26:47.
Skipping row 4099 due to 'NOT_FOUND' error at 2023-10-13 18:30:56.
Skipping row 4223 due to 'NOT_FOUND' error at 2023-10-13 18:31:55.
Skipping row 4829 due to 'NOT_FOUND' error at 2023-10-13 18:37:37.
Skipping row 5521 due to 'NOT_FOUND' error at 2023-10-13 18:43:35

In [ ]:
# MOSTRAR DF

df_tit

In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_tit.to_excel('bd_titular_geo.xlsx', index=False, sheet_name='base')

In [ ]:
query_dependente = f"""
s
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE DEPENDENTES EM DF

df_dep = execute_athena_query(query_dependente)

In [ ]:
# TRATATIVAS NA BASE DE PACIENTES

# REMOVER ACENTOS DAS CIDADES
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_dep['cidade'] = df_dep['cidade'].apply(remove_accents)
df_dep['cidade_clinica'] = df_dep['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_dep['mesma_cidade'] = (df_dep['cidade'] == df_dep['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_dep['faixa_etaria'] = pd.cut(df_dep['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_dep['faixa_utilizacao'] = pd.cut(df_dep['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)


In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_dep['score_recencia'] = df_dep['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_dep['score_frequencia'] = df_dep['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_valor'
df_dep['score_valor'] = df_dep['tm_utilizacao'].map(lambda x: translate_value_score(x))


In [ ]:
# Iterate through the DataFrame
for index, row in df_dep.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_dep.at[index, 'distance_text'] = distance_text
                df_dep.at[index, 'distance_value'] = distance_value
                df_dep.at[index, 'duration_text'] = duration_text
                df_dep.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error at {current_time}.")
            continue


Skipping row 132 due to 'NOT_FOUND' error at 2023-10-14 06:59:23.
Skipping row 230 due to 'NOT_FOUND' error at 2023-10-14 06:59:51.
Skipping row 392 due to 'NOT_FOUND' error at 2023-10-14 07:00:37.
Skipping row 1215 due to 'NOT_FOUND' error at 2023-10-14 07:04:29.
Skipping row 2438 due to 'NOT_FOUND' error at 2023-10-14 07:10:21.
Skipping row 2540 due to 'NOT_FOUND' error at 2023-10-14 07:10:51.
Skipping row 3989 due to 'NOT_FOUND' error at 2023-10-14 07:19:23.
Skipping row 4745 due to 'NOT_FOUND' error at 2023-10-14 07:24:29.
Skipping row 5844 due to 'NOT_FOUND' error at 2023-10-14 07:31:49.
Skipping row 7283 due to 'NOT_FOUND' error at 2023-10-14 07:41:39.
Skipping row 7381 due to 'NOT_FOUND' error at 2023-10-14 07:42:16.
Skipping row 7384 due to 'NOT_FOUND' error at 2023-10-14 07:42:17.
Skipping row 7636 due to 'NOT_FOUND' error at 2023-10-14 07:43:49.
Skipping row 8780 due to 'NOT_FOUND' error at 2023-10-14 07:51:21.
Skipping row 9646 due to 'NOT_FOUND' error at 2023-10-14 07:57:02

In [ ]:
# MOSTRAR DF

df_dep

In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_dep.to_excel('bd_dependente_geo.xlsx', index=False, sheet_name='base')

In [ ]:
query_particular = f"""
sql query
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE PARTICULAR EM DF

df_part = execute_athena_query(query_particular)

In [ ]:
# TRATATIVAS NA BASE DE PACIENTES

# REMOVER ACENTOS DAS CIDADES
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_part['cidade'] = df_part['cidade'].apply(remove_accents)
df_part['cidade_clinica'] = df_part['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_part['mesma_cidade'] = (df_part['cidade'] == df_part['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_part['faixa_etaria'] = pd.cut(df_part['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_part['faixa_utilizacao'] = pd.cut(df_part['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)


In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_part['score_recencia'] = df_part['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_part['score_frequencia'] = df_part['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_valor'
df_part['score_valor'] = df_part['tm_utilizacao'].map(lambda x: translate_value_score(x))


In [ ]:
# Iterate through the DataFrame
for index, row in df_part.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_part.at[index, 'distance_text'] = distance_text
                df_part.at[index, 'distance_value'] = distance_value
                df_part.at[index, 'duration_text'] = duration_text
                df_part.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error at {current_time}.")
            continue


Skipping row 318 due to 'NOT_FOUND' error at 2023-10-14 08:52:01.
Skipping row 1008 due to 'NOT_FOUND' error at 2023-10-14 08:56:23.
Skipping row 1155 due to 'NOT_FOUND' error at 2023-10-14 08:57:20.
Skipping row 1526 due to 'NOT_FOUND' error at 2023-10-14 08:59:39.
Skipping row 1815 due to 'NOT_FOUND' error at 2023-10-14 09:01:31.
Skipping row 2982 due to 'NOT_FOUND' error at 2023-10-14 09:08:49.
Skipping row 3005 due to 'NOT_FOUND' error at 2023-10-14 09:08:57.
Skipping row 3010 due to 'NOT_FOUND' error at 2023-10-14 09:08:59.
Skipping row 3094 due to 'NOT_FOUND' error at 2023-10-14 09:09:32.
Skipping row 3128 due to 'NOT_FOUND' error at 2023-10-14 09:09:46.


In [ ]:
# MOSTRAR DF

df_part

In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_part.to_excel('bd_particular_geo.xlsx', index=False, sheet_name='base')